In [53]:

# import os
# import asyncio
# import pickle
# import pandas as pd
# import streamlit as st
# from dotenv import load_dotenv

# # LangChain integrations
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
# from langchain.agents import initialize_agent, Tool

# # --------------------------
# # Setup
# # --------------------------
# load_dotenv()
# st.title("📊 Marketing Assistant with Churn Prediction")

# google_api_key = os.getenv("GOOGLE_API_KEY")
# chat_model_name = "models/gemini-2.0-flash-lite-preview"

# # Ensure event loop exists
# try:
#     asyncio.get_running_loop()
# except RuntimeError:
#     asyncio.set_event_loop(asyncio.new_event_loop())

# # --------------------------
# # Load your churn model
# # --------------------------
# with open(r"D:\ITI\1_etisalt\Etisalat\churn\models\churn.pkl", "rb") as f:
#     churn_model = pickle.load(f)

# # Define churn prediction tool
# def predict_churn(data: dict) -> str:
#     """
#     Takes a dict of customer features and returns churn prediction.
#     Expected fields:
#     CustomerId, Gender, Senior_Citizen, Is_Married, Dependents, tenure,
#     Phone_Service, Dual, Internet_Service, Online_Security, Online_Backup,
#     Device_Protection, Tech_Support, Streaming_TV, Streaming_Movies,
#     Contract, Paperless_Billing, Payment_Method, Monthly_Charges, Total_Charges
#     """
#     try:
#         df = pd.DataFrame([data])
#         pred = churn_model.predict(df)[0]
#         prob = None
#         if hasattr(churn_model, "predict_proba"):
#             prob = churn_model.predict_proba(df)[0].max()

#         if prob:
#             return f"Prediction: {pred} (confidence: {prob:.2f})"
#         return f"Prediction: {pred}"
#     except Exception as e:
#         return f"❌ Error making prediction: {str(e)}"

# # Wrap model as tool
# tools = [
#     Tool(
#         name="Churn Predictor",
#         func=predict_churn,
#         description=(
#             "Use this tool to predict if a customer will churn. "
#             "Provide customer details as a JSON dict with keys such as "
#             "Gender, Senior_Citizen, Is_Married, Dependents, tenure, "
#             "Phone_Service, Internet_Service, Contract, Monthly_Charges, "
#             "Total_Charges, etc."
#         )
#     )
# ]

# # --------------------------
# # Session state for messages
# # --------------------------
# if "messages" not in st.session_state:
#     st.session_state.messages = []
#     st.session_state.messages.append(SystemMessage(
#         "You are a marketing assistant. "
#         "If the user asks about churn prediction, use the Churn Predictor tool. "
#         "Otherwise, answer normally."
#     ))

# # --------------------------
# # Display previous messages
# # --------------------------
# for message in st.session_state.messages:
#     if isinstance(message, HumanMessage):
#         with st.chat_message("user"):
#             st.markdown(message.content)
#     elif isinstance(message, AIMessage):
#         with st.chat_message("assistant"):
#             st.markdown(message.content)

# # --------------------------
# # Chat input
# # --------------------------
# prompt = st.chat_input("Ask about churn or marketing insights...")

# if prompt:
#     # Add user input
#     with st.chat_message("user"):
#         st.markdown(prompt)
#     st.session_state.messages.append(HumanMessage(prompt))

#     # Create LLM + Agent
#     model_llm = ChatGoogleGenerativeAI(
#         model=chat_model_name,
#         temperature=0,
#         google_api_key=google_api_key
#     )

#     agent = initialize_agent(
#         tools=tools,
#         llm=model_llm,
#         agent="zero-shot-react-description",
#         verbose=True
#     )

#     # Run agent
#     result = agent.run(prompt)

#     # Display response
#     with st.chat_message("assistant"):
#         st.markdown(result)
#     st.session_state.messages.append(AIMessage(result))


In [54]:
import os
import asyncio
import pickle
import pandas as pd
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.agents import initialize_agent, Tool

In [55]:

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
chat_model_name = "models/gemini-2.0-flash-lite-preview"


In [56]:
# Ensure event loop exists
try:
    asyncio.get_running_loop()
except RuntimeError:
    asyncio.set_event_loop(asyncio.new_event_loop())


with open("../models/churn.pkl", "rb") as f:
    churn_model = pickle.load(f)

In [57]:
print(churn_model.named_steps["preprocessor"].feature_names_in_)


['Senior_Citizen' 'Is_Married' 'Dependents' 'tenure' 'Internet_Service'
 'Online_Security' 'Online_Backup' 'Device_Protection' 'Tech_Support'
 'Streaming_TV' 'Streaming_Movies' 'Contract' 'Paperless_Billing'
 'Payment_Method' 'Monthly_Charges' 'Total_Charges']


In [ ]:
def predict_churn(data: dict) -> str:
    """
    Takes a dict of customer features and returns churn prediction
    with confidence score in a descriptive sentence.
    """
    try:
        EXPECTED_COLS = [
            'Senior_Citizen', 'Is_Married', 'Dependents', 'tenure',
            'Internet_Service', 'Online_Security', 'Online_Backup',
            'Device_Protection', 'Tech_Support', 'Streaming_TV',
            'Streaming_Movies', 'Contract', 'Paperless_Billing',
            'Payment_Method', 'Monthly_Charges', 'Total_Charges'
        ]

        # Convert dict -> DataFrame
        one_raw = pd.DataFrame([data])

        # Ensure all expected columns exist
        for col in EXPECTED_COLS:
            if col not in one_raw.columns:
                one_raw[col] = None

        one_raw = one_raw[EXPECTED_COLS]
        one_raw.columns = one_raw.columns.astype(str)

        # Fix numeric columns
        one_raw["Senior_Citizen"] = pd.to_numeric(one_raw["Senior_Citizen"], errors="coerce").fillna(0).astype(int)
        one_raw["tenure"] = pd.to_numeric(one_raw["tenure"], errors="coerce").fillna(0).astype(int)
        one_raw["Monthly_Charges"] = pd.to_numeric(one_raw["Monthly_Charges"], errors="coerce").fillna(0.0)
        one_raw["Total_Charges"] = pd.to_numeric(one_raw["Total_Charges"], errors="coerce").fillna(0.0)

        # Predict
        pred = churn_model.predict(one_raw)[0]

        prob = None
        if hasattr(churn_model, "predict_proba"):
            proba = churn_model.predict_proba(one_raw)[0]
            prob = proba[pred]   # probability of predicted class

        # Map back 0/1 to Yes/No
        target_map = {0: "No", 1: "Yes"}
        pred_label = target_map.get(pred, str(pred))

        if prob is not None:
            return f"✅ After running the churn prediction model, the prediction is: **{pred_label}** with a confidence of **{prob:.2f}**."
        else:
            return f"✅ After running the churn prediction model, the prediction is: **{pred_label}**."

    except Exception as e:
        return f"❌ Error making prediction: {str(e)}"


# def predict_churn(data: dict) -> str:
#     """
#     Takes a dict of customer features and returns churn prediction.
#     Expected fields:
#     'Senior_Citizen', 'Is_Married', 'Dependents', 'tenure',
#     'Internet_Service', 'Online_Security', 'Online_Backup',
#     'Device_Protection', 'Tech_Support', 'Streaming_TV',
#     'Streaming_Movies', 'Contract', 'Paperless_Billing',
#     'Payment_Method', 'Monthly_Charges', 'Total_Charges'
#     """
#     try:
#         # Expected schema
#         EXPECTED_COLS = [
#             'Senior_Citizen', 'Is_Married', 'Dependents', 'tenure',
#             'Internet_Service', 'Online_Security', 'Online_Backup',
#             'Device_Protection', 'Tech_Support', 'Streaming_TV',
#             'Streaming_Movies', 'Contract', 'Paperless_Billing',
#             'Payment_Method', 'Monthly_Charges', 'Total_Charges'
#         ]

#         # Convert dict -> DataFrame
#         one_raw = pd.DataFrame([data])

#         # Ensure all expected columns exist
#         for col in EXPECTED_COLS:
#             if col not in one_raw.columns:
#                 one_raw[col] = None  # placeholder

#         # Keep only expected cols, in correct order
#         one_raw = one_raw[EXPECTED_COLS]

#         # Make sure all column names are strings
#         one_raw.columns = one_raw.columns.astype(str)

#         # Fix numeric columns
#         one_raw["Senior_Citizen"] = pd.to_numeric(one_raw["Senior_Citizen"], errors="coerce").fillna(0).astype(int)
#         one_raw["tenure"] = pd.to_numeric(one_raw["tenure"], errors="coerce").fillna(0).astype(int)
#         one_raw["Monthly_Charges"] = pd.to_numeric(one_raw["Monthly_Charges"], errors="coerce").fillna(0.0)
#         one_raw["Total_Charges"] = pd.to_numeric(one_raw["Total_Charges"], errors="coerce").fillna(0.0)

#         # Predict
#         pred = churn_model.predict(one_raw)[0]
#         prob = None
#         if hasattr(churn_model, "predict_proba"):
#             prob = churn_model.predict_proba(one_raw)[0].max()

#         # Map back 0/1 to Yes/No
#         target_map = {0: "No", 1: "Yes"}
#         pred_label = target_map.get(pred, pred)

#         if prob is not None:
#             return f"Prediction: {pred_label} (confidence: {prob:.2f})"
#         return f"Prediction: {pred_label}"

#     except Exception as e:
#         return f"❌ Error making prediction: {str(e)}"



In [59]:
data = {
  "Senior_Citizen": 0,
  "Is_Married": "Yes",
  "Dependents": "No",
  "tenure": 5,
  "Internet_Service": "DSL",
  "Online_Security": "No",
  "Online_Backup": "Yes",
  "Device_Protection": "No",
  "Tech_Support": "No",
  "Streaming_TV": "Yes",
  "Streaming_Movies": "No",
  "Contract": "Month-to-month",
  "Paperless_Billing": "Yes",
  "Payment_Method": "Electronic check",
  "Monthly_Charges": 75.0,
  "Total_Charges": 350.0,
}


print(predict_churn(data))

✅ After running the churn prediction model, the prediction is: **No** with a confidence of **0.62**.


In [60]:
churn_predic_tool = Tool(
    name="Churn Predictor",
    func=predict_churn,
    description=(
        "Predicts whether a customer is likely to churn (leave the service). "
        "You must provide customer details as a JSON dictionary with the following keys: "
        "Senior_Citizen (0 or 1), Is_Married (Yes/No), Dependents (Yes/No), tenure (numeric), "
        "Internet_Service (No/DSL/Fiber optic), Online_Security (Yes/No), Online_Backup (Yes/No), "
        "Device_Protection (Yes/No), Tech_Support (Yes/No), Streaming_TV (Yes/No), "
        "Streaming_Movies (Yes/No), Contract (Month-to-month/One year/Two year), "
        "Paperless_Billing (Yes/No), Payment_Method (Electronic check/Mailed check/Bank transfer (automatic)/Credit card (automatic)), "
        "Monthly_Charges (numeric), and Total_Charges (numeric)."
    )
)



In [61]:
tools = [
    churn_predic_tool,
    # stats_tool
]

In [62]:
model_llm = ChatGoogleGenerativeAI(
    model=chat_model_name,
    temperature=0,
    google_api_key=google_api_key
)

agent = initialize_agent(
    tools=tools,
    llm=model_llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True

)


In [63]:
query = """Predict churn for the following customer: {
  "Senior_Citizen": 0,
  "Is_Married": "Yes",
  "Dependents": "No",
  "tenure": 5,
  "Internet_Service": "DSL",
  "Online_Security": "No",
  "Online_Backup": "Yes",
  "Device_Protection": "No",
  "Tech_Support": "No",
  "Streaming_TV": "Yes",
  "Streaming_Movies": "No",
  "Contract": "Month-to-month",
  "Paperless_Billing": "Yes",
  "Payment_Method": "Electronic check",
  "Monthly_Charges": 75,
  "Total_Charges": 350
}"""

result = agent.run(query)
print("Assistant:", result)




> Entering new AgentExecutor chain...
I need to use the Churn Predictor tool to predict churn for the given customer data.
Action: Churn Predictor
Action Input: {"Senior_Citizen": 0, "Is_Married": "Yes", "Dependents": "No", "tenure": 5, "Internet_Service": "DSL", "Online_Security": "No", "Online_Backup": "Yes", "Device_Protection": "No", "Tech_Support": "No", "Streaming_TV": "Yes", "Streaming_Movies": "No", "Contract": "Month-to-month", "Paperless_Billing": "Yes", "Payment_Method": "Electronic check", "Monthly_Charges": 75, "Total_Charges": 350}
Observation: ✅ After running the churn prediction model, the prediction is: **Yes** with a confidence of **0.51**.
Thought:I have the churn prediction.
Final Answer: Yes

> Finished chain.
Assistant: Yes


In [64]:
import pandas as pd

one_raw = pd.DataFrame([data])
pred = churn_model.predict(one_raw)[0]

prob = churn_model.predict_proba(one_raw)[0].max()

print(f"Prediction: {pred} (confidence: {prob:.2f})")

Prediction: 0 (confidence: 0.62)


In [65]:
print(churn_model.named_steps["preprocessor"].feature_names_in_)


['Senior_Citizen' 'Is_Married' 'Dependents' 'tenure' 'Internet_Service'
 'Online_Security' 'Online_Backup' 'Device_Protection' 'Tech_Support'
 'Streaming_TV' 'Streaming_Movies' 'Contract' 'Paperless_Billing'
 'Payment_Method' 'Monthly_Charges' 'Total_Charges']
